In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(dotenv_path="../.env")

open_api_key = os.getenv("open_api_key")

openAI_params = {
    'api_key': open_api_key
}
client = OpenAI(**openAI_params)

### File Inputs
There are two ways to provide PDFs to OpenAI Models
1. Base64 Encoded Data
2. File ID: After Uploading it to the /v1/files endpoint

##### Method 1: Uploading it via files API

In [11]:
file = client.files.create(
    file=open("2024-11 Color predictive tool - Cocoa application lab Louviers.pdf", "rb"),
    purpose="user_data"
)
file.id

'file-Mad8HR2YaYvBjZv9BS3tdB'

In [15]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "file",
                    "file": {
                        "file_id": file.id,
                    }
                },
                {
                    "type": "text",
                    "text": "What is the summary of the PDF?",
                },
            ]
        }
    ]
)

print("-"*30, "Details about response", "-"*30, sep='\n')
print(f"Model: {completion.model}")

print("\n", "-"*30, "Details about Results", "-"*30, sep='\n')
print(f"Content Result: {completion.choices[0].message.content}")


print("\n", "-"*30, "Details about Usage", "-"*30, sep='\n')
print(f"Result: {completion.usage.to_dict()}")

------------------------------
Details about response
------------------------------
Model: gpt-4o-mini-2024-07-18


------------------------------
Details about Results
------------------------------
Content Result: The PDF outlines the creation of a color predictive tool for cocoa applications, with a focus on improving customer response times and accuracy in predicting cocoa blends. 

Key points include:

1. **Purpose**: 
   - Save time and enhance responsiveness to customer inquiries.
   - Support R&D teams and cost-saving studies.
   
2. **Development**:
   - A digital tool will be developed for applications like beverages and compounds.
   - A database of color values in applications will be created.

3. **Methodology**:
   - A Design of Experiment (DoE) will be conducted to model the impact of different cocoa powders on color and taste.
   - Daily lab analyses will complement this data.

4. **Applications**:
   - Two separate models are developed: one for beverages and another f

#### Method 2: Base64-encoded files

In [30]:
import base64

with open("test_file.pdf", "rb") as f:
    data = f.read()

# data = b"Hello"
base64_string = base64.b64encode(data).decode("utf-8")

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "file",
                    "file": {
                        "file_name": "2024-11 Color predictive tool - Cocoa application lab Louviers.pdf",
                        "file_data": base64_string,
                    }
                },
                {
                    "type": "input_text",
                    "text": "What is the first dragon in the book?",
                }
            ],
        },
    ],
)

print("-"*30, "Details about response", "-"*30, sep='\n')
print(f"Model: {completion.model}")

print("\n", "-"*30, "Details about Results", "-"*30, sep='\n')
print(f"Content Result: {completion.choices[0].message.content}")


print("\n", "-"*30, "Details about Usage", "-"*30, sep='\n')
print(f"Result: {completion.usage.to_dict()}")

BadRequestError: Error code: 400 - {'error': {'message': 'Invalid chat format. Content blocks are expected to be either text or image_url type.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

#### Usage Considerations
1. Token Usage: To help models understand PDF content, we put into the model's context both extracted text and an image of each page—regardless of whether the page includes images. Before deploying your solution at scale, ensure you understand the pricing and token usage implications of using PDFs as input.

2. File size limitations: You can upload up to 100 pages and 32MB of total content in a single request to the API, across multiple file inputs.